<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install -qU langchain-pinecone pinecone-notebooks requests langchain-groq langchain_huggingface

## Configuração de Variaveis

In [28]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

### Geração de embeddings e indíces do banco vetorial

In [29]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [30]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [82]:
import time

index_name = "index-ppc-markdown-ids"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

## Vector store por Inference API do Hugging Face (lento)

In [ ]:
# import requests
# from langchain_pinecone import PineconeVectorStore

# class CustomEmbedding:
#     def __init__(self):
#         self.hf_token = settings.HF_TOKEN
#         self.model_id = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
#         self.api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{self.model_id}"
#         self.headers = {"Authorization": f"Bearer {self.hf_token}"}

#     def embed_query(self, query):
#         return self.get_embeddings([query])[0]

#     def embed_documents(self, documents):
#         return self.get_embeddings(documents)

#     def get_embeddings(self, texts):
#         response = requests.post(self.api_url, headers=self.headers, json={"inputs": texts, "options": {"wait_for_model": True}})
#         response.raise_for_status()  # Raise an error for bad responses
#         return response.json()

# def initialize_vector_store(index):
#     # Initialize the custom embedding class
#     custom_embedding = CustomEmbedding()

#     # Initialize PineconeVectorStore with the custom embedding class
#     vector_store = PineconeVectorStore(index=index, embedding=custom_embedding)

#     return vector_store


In [ ]:
# vector_store = initialize_vector_store(index)
# results = vector_store.similarity_search("Limite de Atividades de ensino", k=2)

## Vector store local

In [32]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

In [83]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Geração da resposta

In [222]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=4
)



def query_transformation(query):

    query_transformation_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""
    Esse é minha pergunta: {query}

    Analise a pergunta e siga as seguintes regras:

    1. Se a pergunta for sobre dados que parecem ser tabulares, como ementas, limite de horas, fichas de avaliação, atividades ou componentes curriculares, transforme a pergunta em uma string concisa usando a estrutura de tabela markdown. Exemplo: "Qual a ementa de Teoria da Computação?" deve ser transformada para "TEORIA DA COMPUTAÇÃO | EMENTA".

    2. Se a pergunta for mais direta e não se enquadrar na categoria acima, apenas adapte a pergunta para uma versão mais concisa, sem transformá-la em uma estrutura de tabela markdown.

    3. Desfaça siglas antes de retornar a string, da seguinte maneira:
       - PPC = Plano Pedagógico do Curso
       - ACC = Atividades Curriculares Complementares
       - ORG = Organização de Computadores
       - BD = Banco de Dados
       - ENG = Engenharia
       - PGP = Planejamento e Gestão de Projetos
       - TCC = Trabalho de Conclusão de Curso

    4. Não coloque o termo em siglas, caso venha completo.

    Retorne apenas a string sem nenhuma frase introdutória.
    """

    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    chain = prompt_template | query_transformation_llm
    response = chain.invoke({})
    return response.content



def get_context(query, source_name = 'ppc2024', ids = []):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    page_content = '-' * 20 + '\n\n'.join([result.page_content for result in results if result.metadata['id'] not in ids])
    ids = [result.metadata['id'] for result in results if result.metadata['id'] not in ids]
    return page_content, ids



def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua função é fornecer respostas precisas e confiáveis sobre qualquer questão relacionada ao PPC, utilizando apenas as informações contidas no documento como referência. "
                "Se a resposta para pergunta não estiver nos trechos fornecidos, simplesmente informe que a resposta não foi encontrada."
                "\n\n"
                "Responda exclusivamente em português brasileiro e formate a resposta em markdown simples. Sempre referencie o PPC como a fonte de suas respostas."
            ),
            (
                "human",
                f"Dado os seguintes trechos do PPC: {contexto}, responda a pergunta: {pergunta}.\n\n.Lembre-se de utilizar apenas o PPC como fonte para responder. Lembre de citar o trecho ou seção de onde recuperou as informações. Respire fundo e valide a resposta. "
            ),
        ]
    )
    return prompt



def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    print(response)
    return response


def get_answer(query, source_name):
    expanded_query = query_transformation(query)
    print('Query expandida: ', expanded_query)

    query_context, ids = get_context(query, source_name)
    print(len(ids))
    expanded_query_context, new_ids = get_context(expanded_query, source_name, ids)
    print(len(new_ids))
    context = query_context + '\n\n' + expanded_query_context

    prompt = build_prompt(context, query)
    response = get_completion(prompt)


    return context, response.content, expanded_query


In [216]:
context, answer, _ = get_answer('Quais são as Modalidades de condução do tcc', 'ppc2018')
answer

Query expandida:  Quais são as Modalidades de condução do Trabalho de Conclusão de Curso 

5
4
content='De acordo com o Artigo 8º do Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS, o aluno pode escolher uma das modalidades listadas em uma tabela para a condução dos trabalhos do Trabalho de Conclusão de Curso (TCC). No entanto, o trecho fornecido não especifica qual é o conteúdo dessa tabela. Portanto, não é possível responder à pergunta sem uma referência adicional à tabela em questão.' response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 2399, 'total_tokens': 2533, 'completion_time': 0.215287385, 'prompt_time': 0.129824709, 'queue_time': 0.0012071099999999835, 'total_time': 0.345112094}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-52256966-6867-4e9b-aff5-653984216f4a-0' usage_metadata={'input_tokens': 2399, 'output_tokens': 134, 'total_tokens': 2533}


'De acordo com o Artigo 8º do Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS, o aluno pode escolher uma das modalidades listadas em uma tabela para a condução dos trabalhos do Trabalho de Conclusão de Curso (TCC). No entanto, o trecho fornecido não especifica qual é o conteúdo dessa tabela. Portanto, não é possível responder à pergunta sem uma referência adicional à tabela em questão.'

In [43]:
answer

'Sim, é possível utilizar um artigo científico como TCC (Trabalho de Conclusão de Curso), conforme estabelecido no Artigo 15 do contexto fornecido. No entanto, há algumas condições a serem atendidas:\n\n- O artigo deve ser diagramado em formato de conferência ou periódico reconhecido pela SBC (Sociedade Brasileira de Computação), na estrutura de duas colunas, com no mínimo 8 (oito) páginas.\n- O proponente do TCC deve ser um dos autores do artigo científico.\n- O artigo deve ser endossado pelo professor-orientador do TCC.\n- O artigo deve ser integrado ao acervo de trabalhos digitais da UFFS, respeitando os termos de direitos autorais em vigor.\n- Caso o artigo científico seja aceito para publicação em periódico ou conferência reconhecidos com Qualis na área antes da data de sua apresentação à banca, as obrigações do §1º do Art. 11 do contexto estarão suspensas.\n\nAlém disso, é importante respeitar as normas metodológicas preconizadas pela UFFS (Universidade Federal da Fronteira Sul) 

## Perguntas

In [163]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [224]:
!mkdir -p data
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2018.csv data/
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2024.csv data/


In [226]:
import pandas as pd
import csv
import numpy as np
from time import sleep

def process_csv_and_generate_answers(input_csv_path, output_csv_path, inference_config, source_name = 'ppc2024'):


    df = pd.read_csv(input_csv_path)

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ["Pergunta", "RespostaIdeal", "RespostaGerada", "ConsultaExpandida", "ConfiguracaoDeInferencia"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()

        for index, row in df.iterrows():
            question = row['Pergunta']

            goal_response = row['Resposta'] if not pd.isna(row['Resposta']) else ''

            context, response, expanded_query = get_answer(question, source_name)

            writer.writerow({
                "Pergunta": question,
                "RespostaIdeal": goal_response,
                "RespostaGerada": response,
                "ConsultaExpandida": expanded_query,
                "ConfiguracaoDeInferencia": inference_config
            })

            print(f"Pergunta: {question}")
            print(f"Resposta: {response}")
            sleep(20)

    print(f"Novo CSV salvo com sucesso em {output_csv_path}.")


In [181]:
# Exemplo de uso
ppc_2018_input_csv_path = 'data/GroundTruthPPC2018.csv'
ppc_2018_output_csv_path = 'QA_ppc2018.csv'
inference_config = "Mixtral e Gemma 9b(Transformação). Contexto: 5, 5 --> Original e expandida"

process_csv_and_generate_answers(ppc_2018_input_csv_path, ppc_2018_output_csv_path, inference_config, 'ppc2018')

Query expandida:  INFORMATICA BASICA | EMENTA 
Qual é a ementa de Informática Básica?
5
2
content='A ementa da disciplina Informática Básica, conforme o Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS, é:\n\n> Introdução aos conceitos básicos de informática, algoritmos, estruturas de dados, programação estruturada e paradigmas de programação. [Fonte: PPC, p. 33, Código GEX208]' response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 3673, 'total_tokens': 3785, 'completion_time': 0.181944105, 'prompt_time': 0.341621769, 'queue_time': 0.049658679999999955, 'total_time': 0.523565874}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-f7cda3c9-e43a-464a-9f84-39a0bd995d0c-0' usage_metadata={'input_tokens': 3673, 'output_tokens': 112, 'total_tokens': 3785}
Pergunta: Qual é a ementa de Informática Básica?
Resposta: A ementa da disciplina Informática Básica, conforme o Plano Pedag

In [227]:
# Exemplo de uso
ppc_2024_input_csv_path = 'data/GroundTruthPPC2024.csv'
ppc_2024_output_csv_path = 'QA_ppc2024.csv'
inference_config = "Mixtral e Gemma 9b(Transformação). Contexto: 5, 5 --> Original e expandida"

process_csv_and_generate_answers(ppc_2024_input_csv_path, ppc_2024_output_csv_path, inference_config, 'ppc2024')

Query expandida:  INTRODUÇÃO À COMPUTAÇÃO | EMENTA 

5
5
content='A ementa do componente curricular "Introdução à Computação" não está explicitamente fornecida no trecho do Plano Pedagógico do Curso (PPC) que me foi fornecido. No entanto, podemos inferir o conteúdo dessa disciplina a partir do seu nome e da análise dos créditos e horas associadas a ela.\n\nNo PPC, a disciplina "Introdução à Computação" tem o código GEX112, pertence à 1ª fase do curso, tem duração de 4 créditos e é composta por 45 horas de aulas teóricas, 9 horas de aulas práticas e 6 horas de outras atividades, totalizando 60 horas totais.\n\nConsiderando essas informações, podemos supor que a disciplina "Introdução à Computação" tem como objetivo fornecer uma visão geral sobre os conceitos básicos da computação, abordando temas como história da computação, arquitetura de computadores, sistemas operacionais, programação e redes de computadores. Esses tópicos devem ser abordados de forma a proporcionar uma base sólida p

In [182]:
import shutil
from datetime import datetime
import pytz

def save_csv_to_drive(csv_file_path, drive_folder_path):
    # Define the timezone for BRT
    brt_timezone = pytz.timezone('America/Sao_Paulo')

    file_name = csv_file_path.split('.csv')[0]

    # Generate a timestamp in BRT timezone
    timestamp = datetime.now(brt_timezone).strftime('%d_%m_%Y_%H-%M-%S')

    # Define the path in Google Drive with timestamp
    drive_path = f'{drive_folder_path}/{file_name}_{timestamp}.csv'

    # Copy the CSV file to Google Drive
    shutil.copy(csv_file_path, drive_path)
    print(f'File saved to: {drive_path}')



In [183]:
csv_file_path = 'QA_ppc2018.csv'
drive_folder_path = '/content/drive/MyDrive/tcc/resultados'
save_csv_to_drive(csv_file_path, drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/QA_ppc2018_01_09_2024_19-48-27.csv


In [228]:
save_csv_to_drive('QA_ppc2024.csv', drive_folder_path)


File saved to: /content/drive/MyDrive/tcc/resultados/QA_ppc2024_01_09_2024_21-41-48.csv
